In [9]:
# Cell 1: Imports
import numpy as np
from pathlib import Path

from src.core.layers.landfire import LandfireLayer
from src.core.loaded_layer import LoadedLayer
from src.core.grid import Grid
from src.config import SAMPLE_001_BOUNDS_LATLON, LANDFIRE_VARIABLES, CHANNEL_ORDER

ImportError: cannot import name 'LoadedLayer' from 'src.core.loaded_layer' (/Users/alvarogalvan/Projects/Wildfire-Neural-Cellular-Automata/src/core/loaded_layer.py)

In [8]:
# Cell 0: Setup Python path
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")

Project root: /Users/alvarogalvan/Projects/Wildfire-Neural-Cellular-Automata


In [ ]:
# Cell 2: Setup paths
data_dir = Path("data/raw/sample_001/landfire")

# Check if data exists
if not data_dir.exists():
    print(f"Error: Data directory not found at {data_dir}")
else:
    print(f"Data directory found: {data_dir}")
    print(f"Files: {list(data_dir.glob('*.tif'))}")

In [ ]:
# Cell 3: Load a single Landfire layer (CBD as example)
print("Loading CBD (Canopy Bulk Density)...")
cbd_path = data_dir / LANDFIRE_VARIABLES['cbd']
cbd_layer = LandfireLayer(str(cbd_path), 'cbd')

print(f"Layer: {cbd_layer}")
print(f"Layer type: {cbd_layer.get_layer_type()}")

In [ ]:
# Cell 4: Convert to LoadedLayer
cbd_loaded = cbd_layer.to_loaded_layer()
print(f"LoadedLayer: {cbd_loaded}")
print(f"Shape: {cbd_loaded.shape}")
print(f"CRS: {cbd_loaded.crs}")
print(f"Bounds: {cbd_loaded.bounds}")

In [ ]:
# Cell 5: Get statistics
stats = cbd_loaded.get_stats()
print("\nCBD Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

In [ ]:
# Cell 6: Quick visualization of raw CBD data
plt.figure(figsize=(10, 8))
plt.imshow(cbd_loaded.data, cmap='YlGn', interpolation='nearest')
plt.colorbar(label='kg/m³')
plt.title('CBD (Canopy Bulk Density) - Raw Data')
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Cell 7: Create Grid
print("\nCreating Grid...")
# Note: We need to get bounds in the correct CRS
# For now, let's use the bounds from the first loaded layer
grid = Grid(
    bounds=cbd_loaded.bounds,
    resolution=30,
    crs=str(cbd_loaded.crs)
)

print(f"Grid: {grid}")
print(f"Grid shape: {grid.shape}")


In [ ]:
# Cell 8: Add CBD to grid
print("\nAdding CBD to grid...")
grid.add_layer(cbd_loaded, 'cbd')
grid.summary()

In [ ]:
# Cell 9: Load and add all Landfire continuous variables
print("\nLoading all continuous Landfire variables...")
continuous_vars = ['cbd', 'cbh', 'cc', 'ch']

for var_name in continuous_vars:
    if var_name == 'cbd':
        continue  # Already loaded
    
    print(f"Loading {var_name}...")
    file_path = data_dir / LANDFIRE_VARIABLES[var_name]
    layer = LandfireLayer(str(file_path), var_name)
    loaded = layer.to_loaded_layer()
    grid.add_layer(loaded, var_name)

grid.summary()

In [ ]:
# Cell 10: Load and add categorical variables
print("\nLoading categorical Landfire variables...")
categorical_vars = ['fbfm40', 'dist']

for var_name in categorical_vars:
    print(f"Loading {var_name}...")
    file_path = data_dir / LANDFIRE_VARIABLES[var_name]
    layer = LandfireLayer(str(file_path), var_name)
    loaded = layer.to_loaded_layer()
    
    # Show unique codes before mapping
    unique_codes = loaded.get_unique_values()
    print(f"  {var_name} unique codes: {len(unique_codes)} values")
    print(f"  Sample codes: {unique_codes[:10]}")
    
    grid.add_layer(loaded, var_name)

grid.summary()

In [ ]:
# Cell 11: Inspect categorical mappings
print("\nCategorical Mappings:")
for channel_name, mapping in grid.categorical_mappings.items():
    print(f"\n{channel_name}:")
    print(f"  Number of categories: {len(mapping)}")
    print(f"  Mapping (code -> index):")
    for code, idx in sorted(mapping.items())[:10]:
        print(f"    {code} -> {idx}")
    if len(mapping) > 10:
        print(f"    ... ({len(mapping) - 10} more)")

In [ ]:
# Cell 12: Visualize all loaded channels
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

loaded_vars = ['cbd', 'cbh', 'cc', 'ch', 'fbfm40', 'dist']

for idx, var_name in enumerate(loaded_vars):
    ax = axes[idx]
    data = grid.get_channel(var_name)
    
    metadata = grid.channel_metadata[var_name]
    is_categorical = metadata['type'] == 'categorical'
    
    if is_categorical:
        im = ax.imshow(data, cmap='tab20', interpolation='nearest')
        title = f"{var_name} (categorical)"
    else:
        im = ax.imshow(data, cmap='viridis', interpolation='nearest')
        title = f"{var_name} ({metadata['units']})"
    
    ax.set_title(title)
    ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

In [ ]:
# Cell 13: Query values at a specific location
print("\nQuerying values at a specific location...")
# Get center point of the grid
minx, miny, maxx, maxy = grid.bounds
center_x = (minx + maxx) / 2
center_y = (miny + maxy) / 2

values = grid.get_values_at(center_x, center_y)
print(f"Values at ({center_x:.2f}, {center_y:.2f}):")
for channel, value in values.items():
    if channel in grid.loaded_channels:
        print(f"  {channel}: {value:.2f}")

In [ ]:
# Cell 14: Save grid
output_path = "data/processed/sample_001_landfire.npz"
grid.save(output_path)
print(f"\nGrid saved to {output_path}")

In [ ]:
# Cell 15: Test loading
print("\nTesting load...")
loaded_grid = Grid.load(output_path)
print(f"Loaded grid: {loaded_grid}")
loaded_grid.summary()

In [ ]:
# Cell 16: Compare original and loaded
print("\nComparing original and loaded grids...")
print(f"Shapes match: {np.array_equal(grid.shape, loaded_grid.shape)}")
print(f"Data match: {np.allclose(grid.data, loaded_grid.data, equal_nan=True)}")
print(f"Loaded channels match: {grid.loaded_channels == loaded_grid.loaded_channels}")